## Model Checking Algo 

In [4]:
import itertools

class Symbol:
    def __init__(self, name):
        self.name = name

    def __repr__(self):
        return self.name

class Not:
    def __init__(self, operand):
        self.operand = operand

    def evaluate(self, model):
        return not evaluate(self.operand, model)

    def __repr__(self):
        return f"¬{self.operand}"

class And:
    def __init__(self, *conjuncts):
        self.conjuncts = conjuncts

    def evaluate(self, model):
        return all(evaluate(c, model) for c in self.conjuncts)

    def __repr__(self):
        return "(" + " ∧ ".join(map(str, self.conjuncts)) + ")"

class Or:
    def __init__(self, *disjuncts):
        self.disjuncts = disjuncts

    def evaluate(self, model):
        return any(evaluate(d, model) for d in self.disjuncts)

    def __repr__(self):
        return "(" + " ∨ ".join(map(str, self.disjuncts)) + ")"

class Implication:
    def __init__(self, antecedent, consequent):
        self.antecedent = antecedent
        self.consequent = consequent

    def evaluate(self, model):
        return (not evaluate(self.antecedent, model)) or evaluate(self.consequent, model)

    def __repr__(self):
        return f"({self.antecedent} → {self.consequent})"

def evaluate(expr, model):
    if isinstance(expr, Symbol):
        return model.get(expr.name, False)
    return expr.evaluate(model)

def symbols(expr):
    if isinstance(expr, Symbol):
        return {expr}
    if isinstance(expr, (Not, And, Or, Implication)):
        result = set()
        for sub in expr.__dict__.values():
            if isinstance(sub, tuple):
                for s in sub:
                    result |= symbols(s)
            elif isinstance(sub, (Symbol, Not, And, Or, Implication)):
                result |= symbols(sub)
        return result
    return set()

def model_check(knowledge, query):
    syms = list(symbols(And(knowledge, query)))
    for values in itertools.product([True, False], repeat=len(syms)):
        model = dict(zip([s.name for s in syms], values))
        if knowledge.evaluate(model):
            if not query.evaluate(model):
                return False
    return True


In [6]:
from logic import *

rain = Symbol("rain")
hagrid = Symbol("hagrid")
dumbledore = Symbol("dumbledore")

knowledge = And(
    Implication(Not(rain), hagrid),
    Or(hagrid, dumbledore),
    Not(And(hagrid, dumbledore)),
    dumbledore
)

print("Does KB entail rain?")
print(model_check(knowledge, rain))


ModuleNotFoundError: No module named 'logic'